# This is breast cancer classification project
### It id devided in several parts:
* Data reading and cleaning
* [EDA](#EDA)
* Visualizations
* Models

In [2]:
# some imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

# Data reading and cleaning

In [11]:
cancer_raw_data = pd.read_csv("SEER Breast Cancer Dataset .csv")
# there are mistake in data, so it has fully-nulled column, so we drop it
cancer_raw_data = cancer_raw_data.drop('Unnamed: 3', axis = 1)

In [12]:
# display head of dataset
cancer_raw_data.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,43,"Other (American Indian/AK Native, Asian/Pacifi...",Married (including common law),T2,N3,IIIC,Moderately differentiated; Grade II,Regional,40,Positive,Positive,19,11,1,Alive
1,47,"Other (American Indian/AK Native, Asian/Pacifi...",Married (including common law),T2,N2,IIIA,Moderately differentiated; Grade II,Regional,45,Positive,Positive,25,9,2,Alive
2,67,White,Married (including common law),T2,N1,IIB,Poorly differentiated; Grade III,Regional,25,Positive,Positive,4,1,2,Dead
3,46,White,Divorced,T1,N1,IIA,Moderately differentiated; Grade II,Regional,19,Positive,Positive,26,1,2,Dead
4,63,White,Married (including common law),T2,N2,IIIA,Moderately differentiated; Grade II,Regional,35,Positive,Positive,21,5,3,Dead


In [14]:
# info on types
cancer_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   Grade                   4024 non-null   object
 7   A Stage                 4024 non-null   object
 8   Tumor Size              4024 non-null   int64 
 9   Estrogen Status         4024 non-null   object
 10  Progesterone Status     4024 non-null   object
 11  Regional Node Examined  4024 non-null   int64 
 12  Reginol Node Positive   4024 non-null   int64 
 13  Survival Months         4024 non-null   int64 
 14  Status                  4024 non-null   object
dtypes: i

In [32]:
for column in cancer_raw_data.select_dtypes(include=['object']):
    print(column)
    print(cancer_raw_data[column].unique(), '\n')

Race 
['Other (American Indian/AK Native, Asian/Pacific Islander)' 'White'
 'Black'] 

Marital Status
['Married (including common law)' 'Divorced' 'Single (never married)'
 'Widowed' 'Separated'] 

T Stage 
['T2' 'T1' 'T3' 'T4'] 

N Stage
['N3' 'N2' 'N1'] 

6th Stage
['IIIC' 'IIIA' 'IIB' 'IIA' 'IIIB'] 

Grade
['Moderately differentiated; Grade II' 'Poorly differentiated; Grade III'
 'Well differentiated; Grade I' 'Undifferentiated; anaplastic; Grade IV'] 

A Stage
['Regional' 'Distant'] 

Estrogen Status
['Positive' 'Negative'] 

Progesterone Status
['Positive' 'Negative'] 

Status
['Alive' 'Dead'] 



In [44]:
print(' MENU '.center(126, '*'))

************************************************************ MENU ************************************************************


In [13]:
cancer_raw_data.describe()

,Age,Tumor Size,Regional Node Examined,Reginol Node Positive,Survival Months
count,4024.000000,4024.000000,4024.000000,4024.000000,4024.000000
mean,53.972167,30.473658,14.357107,4.158052,71.297962
std,8.963134,21.119696,8.099675,5.109331,22.921430
min,30.000000,1.000000,1.000000,1.000000,1.000000
25%,47.000000,16.000000,9.000000,1.000000,56.000000
50%,54.000000,25.000000,14.000000,2.000000,73.000000
75%,61.000000,38.000000,19.000000,5.000000,90.000000
max,69.000000,140.000000,61.000000,46.000000,107.000000


# EDA